<a href="https://colab.research.google.com/github/paroonk/optimization-mip-project/blob/main/ceylie_cincera_youtube_gekko.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Import


In [1]:
!pip install gekko
import numpy as np
from gekko import GEKKO

     |████████████████████████████████| 10.8MB 7.2MB/s 


# Transportation Problem

In [105]:
# transportation problem
# minimize bushel*miles
grove_name = ['Mt.Dara', 'Eustis', 'Clermont']
GROVE = range(len(grove_name))
supply = [275000, 400000, 300000] # amount each grove will supply

plant_name = ['Ocala', 'Orlando', 'Leesburg']
PLANT = range(len(plant_name))
mship = [200000, 600000, 225000] # max amount that can be shipped to each plant

bush = [[21, 50, 40], [35, 30, 22], [55, 20, 25]] # bushel miles for all groves and plants (cost)

m = GEKKO()

### VARIABLES
x = [[m.Var(integer=True, lb=0, ub=None) for j in PLANT] for i in GROVE]


### OBJECTIVE
m.Obj(sum(bush[i][j] * x[i][j] for j in PLANT for i in GROVE))


### CONSTRAINTS
# grove supply not exceed limit
for i in GROVE:
    m.Equation(sum(x[i][j] for j in PLANT) == supply[i])
    
# max amount shipped to each plant not exceed limit
for j in PLANT:
    m.Equation(sum(x[i][j] for i in GROVE) <= mship[j])


### OPTIMIZING
m.options.SOLVER = 1
# m.solve()
m.solve(disp=False)


### RESULTS
print('Objective: ' + str(m.options.objfcnval))
for i in GROVE:
    for j in PLANT:
        val = x[i][j].value[0]
        # if abs(val) > 1e-6: # only printing non-zeros
        print('{} -> {} : {}'.format(grove_name[i], plant_name[j], val))

print('\nCheck results')
for i in GROVE:
    print('{} supply: {} == {}'.format(grove_name[i], sum(x[i][j].value[0] for j in PLANT), supply[i]))
    
for j in PLANT:
    print('{} shipped: {} <= {}'.format(plant_name[j], sum(x[i][j].value[0] for i in GROVE), mship[j]))

Objective: 24000000.0
Mt.Dara -> Ocala : 200000.0
Mt.Dara -> Orlando : 0.0
Mt.Dara -> Leesburg : 75000.0
Eustis -> Ocala : 0.0
Eustis -> Orlando : 250000.0
Eustis -> Leesburg : 150000.0
Clermont -> Ocala : 0.0
Clermont -> Orlando : 300000.0
Clermont -> Leesburg : 0.0

Check results
Mt.Dara supply: 275000.0 == 275000
Eustis supply: 400000.0 == 400000
Clermont supply: 300000.0 == 300000
Ocala shipped: 200000.0 <= 200000
Orlando shipped: 550000.0 <= 600000
Leesburg shipped: 225000.0 <= 225000


# Facility Location Problem

In [106]:
# facility location problem
# minimize activation cost + tranportation cost
facility_name = ['FAC1', 'FAC2', 'FAC3']
FACILITY = range(len(facility_name))
act_cost = [1000, 1000, 1000] # cost to build facility
max_supply = [500, 500, 500] # max supply limit for each facility

customer_name = [1, 2, 3, 4, 5]
CUSTOMER = range(len(customer_name))
demand = [80, 270, 250, 160, 180] # demand for each customer

tran_cost = [[4, 5, 6, 8, 10], [6, 4, 3, 5, 8], [9, 7, 4 ,3, 4]] # transportation cost facility->customer

m = GEKKO()

### VARIABLES
x = [[m.Var(lb=0, ub=None, integer=True) for j in CUSTOMER] for i in FACILITY]
y = [m.Var(lb=0, ub=1, integer=True) for i in FACILITY]


### OBJECTIVE
m.Obj(sum(act_cost[i] * y[i] for i in FACILITY) + sum(tran_cost[i][j] * x[i][j] for j in CUSTOMER for i in FACILITY))


### CONSTRAINTS
for i in FACILITY:
    m.Equation(sum(x[i][j] for j in CUSTOMER) <= max_supply[i] * y[i]) # facility supply not exceed limit (y=1, if need)
    
for j in CUSTOMER:
    m.Equation(sum(x[i][j] for i in FACILITY) == demand[j]) # customer amount not exceed limit
    

### OPTIMIZING
m.options.SOLVER = 1
# m.solve()
m.solve(disp=False)
    

### RESULTS
print('Objective: ' + str(m.options.objfcnval))
for i in FACILITY:
    for j in CUSTOMER:
        val = round(x[i][j].value[0], 2)
        if abs(val) > 1e-6: # only printing non-zeros
            print('{}->{} : {}'.format(facility_name[i], customer_name[j], val))

print('\nCheck results')
for i in FACILITY:
    print('{} supply: {} <= {}'.format(facility_name[i], sum(x[i][j].value[0] for j in CUSTOMER), max_supply[i] * y[i].value[0]))

for j in CUSTOMER:
    print('{} demand: {} == {}'.format(customer_name[j], sum(x[i][j].value[0] for i in FACILITY), demand[j]))

Objective: 5610.0
FAC2->1 : 80.0
FAC2->2 : 270.0
FAC2->3 : 150.0
FAC3->3 : 100.0
FAC3->4 : 160.0
FAC3->5 : 180.0

Check results
FAC1 supply: 0.0 <= 0.0
FAC2 supply: 500.0 <= 500.0
FAC3 supply: 440.0 <= 500.0
1 demand: 80.0 == 80
2 demand: 270.0 == 270
3 demand: 250.0 == 250
4 demand: 160.0 == 160
5 demand: 180.0 == 180


# Fire Station Problem

In [110]:
# fire station problem
# minimize number of fire station build

city_location = [1, 2, 3, 4, 5, 6]
CITY = range(len(city_location))

city_distance = [[0, 10, 20, 30, 30, 20], [10, 0, 25, 35, 20, 10], [20, 25, 0, 15, 30, 20], [30, 35, 15, 0, 15, 25], [20, 20, 30, 15, 0, 14], [20, 10, 20, 25, 14, 0]] # distance from city->city
max_distance = 15 # distance between city to nearest city which have fire station must not exceed 15

m = GEKKO()

### VARIABLES
x = [m.Var(integer=True, lb=0, ub=None) for i in CITY]


### OBJECTIVE
m.Obj(sum(x[i] for i in CITY))


### CONSTRAINTS
for i in CITY:
    m.Equation(sum(x[j] for j in CITY if city_distance[i][j] <= 15) >= 1)
    

### OPTIMIZING
m.options.SOLVER = 1
# m.solve()
m.solve(disp=False)


### RESULTS
print('Objective: ' + str(m.options.objfcnval))
for i in CITY:
    print('City {} : {}'.format(city_location[i], x[i].value[0]))

select_city = [int(x[i].value[0]) for i in CITY]
for i in CITY:
    dist_list = []
    for j in CITY:
        if select_city[j] == 1:
            dist_list.append(0 if j == i else city_distance[i][j])
        else:
            dist_list.append(999)
    dist_list = np.array(dist_list)
    print('City {} : {}, nearest city = City {} ({})'.format(city_location[i], dist_list, city_location[np.argmin(dist_list)], dist_list[np.argmin(dist_list)]))

Objective: 2.0
City 1 : 0.0
City 2 : 1.0
City 3 : 0.0
City 4 : 1.0
City 5 : 0.0
City 6 : 0.0
City 1 : [999  10 999  30 999 999], nearest city = City 2 (10)
City 2 : [999   0 999  35 999 999], nearest city = City 2 (0)
City 3 : [999  25 999  15 999 999], nearest city = City 4 (15)
City 4 : [999  35 999   0 999 999], nearest city = City 4 (0)
City 5 : [999  20 999  15 999 999], nearest city = City 4 (15)
City 6 : [999  10 999  25 999 999], nearest city = City 2 (10)


# Blending Problem

In [142]:
# Blending problem
# minimize cost
# need 2000 kcal, 55 protein, 800 calcium

food_list = ['Oatmeal', 'Chicken', 'Eggs', 'Whole milk', 'Cherry pie', 'Pork and beans']
FOOD = range(len(food_list))
kcal = [110, 205, 160, 160, 420, 260]
protein = [4, 32, 13, 8, 4, 14]
calcium = [2, 12, 54, 285, 22, 80]
price = [0.3, 2.4, 1.3, 0.9, 2.0, 1.9]

req_kcal = 2000
req_protein = 55
req_calcium = 800

m = GEKKO()

### VARIABLES
x = [m.Var(integer=True, lb=0, ub=None) for i in FOOD]


### OBJECTIVE
m.Obj(sum(price[i] * x[i] for i in FOOD))


### CONSTRAINTS
m.Equation(sum(kcal[i] * x[i] for i in FOOD) >= req_kcal)
m.Equation(sum(protein[i] * x[i] for i in FOOD) >= req_protein)
m.Equation(sum(calcium[i] * x[i] for i in FOOD) >= req_calcium)
    

### OPTIMIZING
m.options.SOLVER = 1
# m.solve()
m.solve(disp=False)


### RESULTS
print('Objective: ' + str(m.options.objfcnval))
for i in FOOD:
    print('{} : {}'.format(food_list[i], x[i].value[0]))

print('\nCheck results')
print('Total kcal: {} >= {}'.format(sum(kcal[i] * x[i].value[0] for i in FOOD), req_kcal))
print('Total protein: {} >= {}'.format(sum(protein[i] * x[i].value[0] for i in FOOD), req_protein))
print('Total calcium: {} >= {}'.format(sum(calcium[i] * x[i].value[0] for i in FOOD), req_calcium))

Objective: 6.9
Oatmeal : 14.0
Chicken : 0.0
Eggs : 0.0
Whole milk : 3.0
Cherry pie : 0.0
Pork and beans : 0.0

Check results
Total kcal: 2020.0 >= 2000
Total protein: 80.0 >= 55
Total calcium: 883.0 >= 800


# Fixed-charge Problem

In [141]:
# Fixed-charge problem
# maximize profit

type_list = ['Shirts', 'Shorts', 'Pants']
TYPE = range(len(type_list))

machine_cost = [200, 150, 100] # production cost per week

req_labor = [3, 2, 6] # Labor hour require per unit produced
req_cloth = [4, 3, 4] # Cloth hour require per unit produced

avail_labor = 150
avail_cloth = 160

sales = [12, 8, 15] # sales price per unit
vc = [6, 4, 8] # variable cost per unit

m = GEKKO()

### VARIABLES
x = [m.Var(integer=True, lb=0, ub=None) for i in TYPE]
y = [m.Var(integer=True, lb=0, ub=1) for i in TYPE]


### OBJECTIVE
m.Obj(-1 * (sum((sales[i] - vc[i]) * x[i] for i in TYPE) - sum(machine_cost[i] * y[i] for i in TYPE)))


### CONSTRAINTS
m.Equation(sum(req_labor[i] * x[i] for i in TYPE) <= avail_labor)
m.Equation(sum(req_cloth[i] * x[i] for i in TYPE) <= avail_cloth)

for i in TYPE:
    m.Equation(x[i] <= 10000 * y[i])
    

### OPTIMIZING
m.options.SOLVER = 1
# m.solve()
m.solve(disp=False)


### RESULTS
print('Objective: ' + str(-1 * m.options.objfcnval))
for i in TYPE:
    print('{} production : {}'.format(type_list[i], x[i].value[0]))
for i in TYPE:
    print('Rent {} machinery : {}'.format(type_list[i], y[i].value[0]))
      
# Check
print('\nCheck results')
print('Labor used: {} <= {}'.format(sum(req_labor[i] * x[i].value[0] for i in TYPE), avail_labor))
print('Cloth used: {} <= {}'.format(sum(req_cloth[i] * x[i].value[0] for i in TYPE), avail_cloth))

Objective: 75.0
Shirts production : 0.0
Shorts production : 0.0
Pants production : 25.0
Rent Shirts machinery : 0.0
Rent Shorts machinery : 0.0
Rent Pants machinery : 1.0

Check results
Labor used: 150.0 <= 150
Cloth used: 100.0 <= 160


# Piecewise Linear Problem

In [140]:
# Piecewise linear problem
# maximize profit
# gas 1 must contain >= 50% oil 1, gas 2 must contain >= 60% oil 1
# gas 1 price = 12/gallon, gas 2 price = 14/gallon
# gas 1 current avail = 500 gallon, gas 2 current avail = 1000 gallon
# Can buy additional oil 1 for 1500 gallon (max), 25/gallon for first 500 gallon, 20 for gallon 500-1000, 15 for gallon 1000-1500

gas_type = [1, 2]
GAS = range(len(gas_type))
sales = [12, 14]

oil_type = [1, 2]
OIL = range(len(oil_type))
req_oil1_ratio = [0.5, 0.6]
avail_oil = [500, 1000]

price_name = ['0-500', '500-1000', '1000-1500']
price_num = [500, 500, 500]
price_value = [25, 20, 15]
PRICE = range(len(price_value))
price_name = ['{}-{}'.format(sum(price_num[:i + 1]) - price_num[i], sum(price_num[:i + 1])) for i in PRICE]

m = GEKKO()

### VARIABLES
x = [[m.Var(integer=True, lb=0, ub=None) for j in OIL] for i in GAS]
y = [m.Var(integer=True, lb=0, ub=None) for k in PRICE]
z = [m.Var(integer=True, lb=0, ub=1) for k in PRICE]


### OBJECTIVE
m.Obj(-1 * (sum(sales[i] * x[i][j] for j in OIL for i in GAS) - sum(price_value[k] * y[k] for k in PRICE)))


### CONSTRAINTS
m.Equation(sum(x[i][0] for i in GAS) <= avail_oil[0] + sum(y[k] for k in PRICE))
m.Equation(sum(x[i][1] for i in GAS) <= avail_oil[1])

for k in PRICE:
    m.Equation(y[k] <= price_num[k] * z[k]) # z need to be 1 if y > 0

    if k > 0:
        m.Equation(y[k - 1] >= price_num[k - 1] * z[k]) # y_k-1 need to be 500 if z_k == 1
      
    m.Equation(z[k] <= y[k]) # z need to be 0 if y == 0 (z not in obj, so required this)
    
for i in GAS:
    m.Equation(x[i][0] >= req_oil1_ratio[i] * sum(x[i][j] for j in OIL))


### OPTIMIZING
m.options.SOLVER = 1
# m.solve()
m.solve(disp=False)


### RESULTS
print('Objective: ' + str(-1 * m.options.objfcnval))
for i in GAS:
    for j in OIL:
        print('Amount of oil {} to produce gas {} : {}'.format(oil_type[j], gas_type[i], x[i][j].value[0]))
for k in PRICE:
    print('Amount of oil 1 purchase group {} : {}'.format(price_name[k], y[k].value[0]))
for k in PRICE:
    print('Purchase from group {} : {}'.format(price_name[k], z[k].value[0]))
    
print('\nCheck results')
for j in OIL:
    print('Oil {} used: {} <= {}'.format(j, sum(x[i][j].value[0] for i in GAS), avail_oil[j] + (1500 if j == 0 else 0)))
for i in GAS:
    print('Gas {} oil1 ratio: {} >= {}'.format(i, round(x[i][0].value[0] / sum(x[i][j].value[0] for j in OIL), 2) if x[i][0].value[0] > 0 else 'NA', req_oil1_ratio[i]))

Objective: 12500.0
Amount of oil 1 to produce gas 1 : 0.0
Amount of oil 2 to produce gas 1 : 0.0
Amount of oil 1 to produce gas 2 : 1500.0
Amount of oil 2 to produce gas 2 : 1000.0
Amount of oil 1 purchase group 0-500 : 500.0
Amount of oil 1 purchase group 500-1000 : 500.0
Amount of oil 1 purchase group 1000-1500 : 0.0
Purchase from group 0-500 : 1.0
Purchase from group 500-1000 : 1.0
Purchase from group 1000-1500 : 0.0

Check results
Oil 0 used: 1500.0 <= 2000
Oil 1 used: 1000.0 <= 1000
Gas 0 oil1 ratio: NA >= 0.5
Gas 1 oil1 ratio: 0.6 >= 0.6


# Piecewise Linear Problem 2

In [180]:
# Piecewise linear problem 2
# maximize exposures

# budget = 20000
# IJ exposures per ads, ads 1-6: 10000, 7-10: 3000, 11-15: 2500, 16+: 0
# FS exposures per ads, ads 1-4: 8000, 5-12: 6000, 13-15: 2000, 16+: 0
# ads cost = 1000

mag_name = ['Inside Jocks', 'Family Square']
MAG = range(len(mag_name))

budget = 20000
cost = 1000

exp_num = [[6, 4, 5], [4, 8, 3]]
exp_value = [[10000, 3000, 2500], [8000, 6000, 2000]]
EXP = [range(len(lst)) for lst in exp_value]
exp_name = [['{}-{}'.format(sum(exp_num[i][:j + 1]) - exp_num[i][j] + 1, sum(exp_num[i][:j + 1])) for j in EXP[i]] for i in MAG]

m = GEKKO()

### VARIABLES
x = [[m.Var(integer=True, lb=0, ub=None) for j in EXP[i]] for i in MAG]
y = [[m.Var(integer=True, lb=0, ub=1) for j in EXP[i]] for i in MAG]


### OBJECTIVE
m.Obj(-1 * sum(exp_value[i][j] * x[i][j] for j in EXP[i] for i in MAG))


### CONSTRAINTS
m.Equation(sum(cost * x[i][j] for j in EXP[i] for i in MAG) <= budget)

for i in MAG:
    for j in EXP[i]:
        m.Equation(x[i][j] <= exp_num[i][j] * y[i][j])
        
        if j > 0:
            m.Equation(x[i][j - 1] >= exp_num[i][j - 1] * y[i][j])

        m.Equation(y[i][j] <= x[i][j]) # y need to be 0 if x == 0 (y not in obj, so required this)


### OPTIMIZING
m.options.SOLVER = 1
# m.solve()
m.solve(disp=False)


### RESULTS
print('Objective: ' + str(-1 * m.options.objfcnval))
for i in MAG:
    for j in EXP[i]:
        val = x[i][j].value[0]
        if abs(val) > 1e-6: # only printing non-zeros
            print('No. of ads {} for {} : {}'.format(exp_name[i][j], mag_name[i], val))
for i in MAG:
    for j in EXP[i]:
        val = y[i][j].value[0]
        if abs(val) > 1e-6: # only printing non-zeros
            print('Use ads {} for {} : {}'.format(exp_name[i][j], mag_name[i], val))
    
print('\nCheck results')
for i in MAG:
    print('No. of ads for {}: {}'.format(mag_name[i], sum([x[i][j].value[0] for j in EXP[i]])))
print('Total cost: {} <= {}'.format(sum([1000 * x[i][j].value[0] for j in EXP[i] for i in MAG]), budget))

Objective: 146000.0
No. of ads 1-6 for Inside Jocks : 6.0
No. of ads 7-10 for Inside Jocks : 2.0
No. of ads 1-4 for Family Square : 4.0
No. of ads 5-12 for Family Square : 8.0
Use ads 1-6 for Inside Jocks : 1.0
Use ads 7-10 for Inside Jocks : 1.0
Use ads 1-4 for Family Square : 1.0
Use ads 5-12 for Family Square : 1.0

Check results
No. of ads for Inside Jocks: 8.0
No. of ads for Family Square: 12.0
Total cost: 20000.0 <= 20000


# Lockbox (Payment Process) Problem

In [181]:
# Lockbox (payment process) problem
# minimize cost

region_name = ['West', 'Midwest', 'East', 'South']
REGION = range(len(region_name))
region_payment = [70000, 50000, 60000, 40000] # payment per day

city_name = ['Los Angeles', 'Chicago', 'New York', 'Atlanta']
CITY = range(len(city_name))

dist = [[2, 6, 8, 8], [6, 2, 5, 5], [8, 5, 2, 5], [8, 5, 5, 2]] # days of mailing payment, region -> city

int_rate = 0.2 # annual interest rate for investment
lockbox_cost = 50000 # per city

m = GEKKO()

### VARIABLES
x = [[m.Var(integer=True, lb=0, ub=None) for j in CITY] for i in REGION]
y = [m.Var(integer=True, lb=0, ub=1) for j in CITY]


### OBJECTIVE
m.Obj((int_rate * sum(dist[i][j] * x[i][j] for j in CITY for i in REGION)) + (lockbox_cost * sum(y[j] for j in CITY)))


### CONSTRAINTS
for i in REGION:
    m.Equation(sum(x[i][j] for j in CITY) == region_payment[i])
  
for j in CITY:
    m.Equation(sum(x[i][j] for i in REGION) <= 1000000 * y[j])


### OPTIMIZING
m.options.SOLVER = 1
# m.solve()
m.solve(disp=False)


### RESULTS
print('Objective: ' + str(m.options.objfcnval))
for i in REGION:
    for j in CITY:
        val = x[i][j].value[0]
        if abs(val) > 1e-6: # only printing non-zeros
            print('Amount of payment from {} to {} : {}'.format(region_name[i], city_name[j], val))
for j in CITY:
    val = y[j].value[0]
    if abs(val) > 1e-6: # only printing non-zeros
        print('Build lockbox in {} : {}'.format(city_name[j], val))
    
print('\nCheck results')
for i in REGION:
    print('Total payment from {}: {} == {}'.format(region_name[i], sum([x[i][j].value[0] for j in CITY]), region_payment[i]))

Objective: 242000.0
Amount of payment from West to Los Angeles : 70000.0
Amount of payment from Midwest to New York : 50000.0
Amount of payment from East to New York : 60000.0
Amount of payment from South to New York : 40000.0
Build lockbox in Los Angeles : 1.0
Build lockbox in New York : 1.0

Check results
Total payment from West: 70000.0 == 70000
Total payment from Midwest: 50000.0 == 50000
Total payment from East: 60000.0 == 60000
Total payment from South: 40000.0 == 40000


# Either-Or Problem

In [183]:
# Either-Or problem
# maximize profit

auto_name = ['Compact', 'Midsize', 'Large']
AUTO = range(len(auto_name))
req_steel = [1.5, 3, 5] # tons per unit
req_labor = [30, 25, 40] # hrs per unit
profit = [2000, 3000, 4000] # dollar per unit

avail_steel = 6000
avail_labor = 60000

min_production = 1000 # if produce, need to be at least 1000 cars

m = GEKKO()

### VARIABLES
x = [m.Var(integer=True, lb=0, ub=None, name='Amount of {} production'.format(auto_name[i])) for i in AUTO]
y = [m.Var(integer=True, lb=0, ub=1, name='Build {}'.format(auto_name[i])) for i in AUTO]


### OBJECTIVE
m.Obj(-1 * sum(profit[i] * x[i] for i in AUTO))


### CONSTRAINTS
for i in AUTO:
    m.Equation(sum(req_steel[i] * x[i] for i in AUTO) <= avail_steel)
    m.Equation(sum(req_labor[i] * x[i] for i in AUTO) <= avail_labor)
    
    m.Equation(x[i] >= min_production * y[i])
    m.Equation(x[i] <= 100000 * y[i])


### OPTIMIZING
m.options.SOLVER = 1
# m.solve()
m.solve(disp=False)


### RESULTS
print('Objective: ' + str(-1 * m.options.objfcnval))
for i in AUTO:
    val = x[i].value[0]
    print('Amount of {} production : {}'.format(auto_name[i], val))
for i in AUTO:
    val = y[i].value[0]
    print('Build {} : {}'.format(auto_name[i], val))
    
print('\nCheck results')
for i in AUTO:
    print('Total steel used: {} <= {}'.format(sum([req_steel[i] * x[i].value[0] for i in AUTO]), avail_steel))
    print('Total labor used: {} <= {}'.format(sum([req_labor[i] * x[i].value[0] for i in AUTO]), avail_labor))

Objective: 6000000.0
Amount of Compact production : 0.0
Amount of Midsize production : 2000.0
Amount of Large production : 0.0
Build Compact : 0.0
Build Midsize : 1.0
Build Large : 0.0

Check results
Total steel used: 6000.0 <= 6000
Total labor used: 50000.0 <= 60000
Total steel used: 6000.0 <= 6000
Total labor used: 50000.0 <= 60000
Total steel used: 6000.0 <= 6000
Total labor used: 50000.0 <= 60000


# Multiperiod Work Scheduling

In [184]:
# Multiperiod Work Scheduling
# minimize cost

month_name = ['January', 'February', 'March', 'April', 'May']
MONTH = range(len(month_name))
service_demand = [6000, 7000, 8000, 9500, 11000] # hrs / month

worker_init = 50
worker_output = 160 # time worker can work per month
worker_wage = 2000

# use 1 month to train
train_supervize = 50 # req experienced worker time per month
train_wage = 1000

turnover_rate = 0.05 # worker resign rate

m = GEKKO()

### VARIABLES
x = [m.Var(integer=True, lb=0, ub=None) for i in MONTH]
y = [m.Var(integer=True, lb=0, ub=None) for i in MONTH]


### OBJECTIVE
m.Obj(sum((worker_wage * x[i]) + (train_wage * y[i]) for i in MONTH))


### CONSTRAINTS
for i in MONTH:
    m.Equation((worker_output * x[i]) - (train_supervize * y[i]) >= service_demand[i])
    
    if i == 0:
        m.Equation(x[i] == 50)
        
    else:
        m.Equation(x[i] >= (1 - turnover_rate) * x[i - 1])
        m.Equation(x[i] <= (1 - turnover_rate) * x[i - 1] + y[i - 1])


### OPTIMIZING
m.options.SOLVER = 1
# m.solve()
m.solve(disp=False)


### RESULTS
print('Objective: ' + str(m.options.objfcnval))
for i in MONTH:
    val = x[i].value[0]
    print('No. of worker in month {} : {}'.format(month_name[i], val))
for i in MONTH:
    val = y[i].value[0]
    print('No. of trainee in month {} : {}'.format(month_name[i], val))
        
print('\nCheck results')
for i in MONTH:
    print('No. of worker hrs in month {}: {} >= {}'.format(month_name[i], (worker_output * x[i].value[0]) - (train_supervize * y[i].value[0]), service_demand[i]))

for i in MONTH[1:]:
  print('No. of turnover in month {}: {} >= {}'.format(month_name[i], x[i - 1].value[0] + y[i - 1].value[0] - x[i].value[0], round(turnover_rate * x[i - 1].value[0], 2)))

Objective: 600000.0
No. of worker in month January : 50.0
No. of worker in month February : 48.0
No. of worker in month March : 54.0
No. of worker in month April : 63.0
No. of worker in month May : 69.0
No. of trainee in month January : 1.0
No. of trainee in month February : 9.0
No. of trainee in month March : 12.0
No. of trainee in month April : 10.0
No. of trainee in month May : 0.0

Check results
No. of worker hrs in month January: 7950.0 >= 6000
No. of worker hrs in month February: 7230.0 >= 7000
No. of worker hrs in month March: 8040.0 >= 8000
No. of worker hrs in month April: 9580.0 >= 9500
No. of worker hrs in month May: 11040.0 >= 11000
No. of turnover in month February: 3.0 >= 2.5
No. of turnover in month March: 3.0 >= 2.4
No. of turnover in month April: 3.0 >= 2.7
No. of turnover in month May: 4.0 >= 3.15


# College Apparel

In [193]:
# College Apparel
# maximize profit

shirt_name = ['Jersey', 'Sweatshirt', 'Red', 'Black', 'White']
SHIRT = range(len(shirt_name))
shirt_cost = [11.18, 20.11, 16.99, 16.99, 16.99]

style_name = ['Criss Cross', 'Crop Top', 'Halter', 'Jersey Set', 'Hoodie', 'Shoulder']
STYLE = range(len(style_name))
style_price = [45, 40, 45, 45, 50 ,45]

budget = 300

m = GEKKO()

### VARIABLES
import random
x = [[m.Var(value=random.random(), integer=True, lb=0, ub=None) for j in STYLE] for i in SHIRT]


### OBJECTIVE
m.Obj(-1 * sum((style_price[j] - shirt_cost[i]) * x[i][j] for j in STYLE for i in SHIRT))


### CONSTRAINTS
m.Equation(sum(shirt_cost[i] * x[i][j] for j in STYLE for i in SHIRT) <= budget)

m.Equation(sum(x[i][3] for i in SHIRT if i != 0) == 0)
m.Equation(sum(x[i][4] for i in SHIRT if i != 1) == 0)
m.Equation(sum(x[0][j] for j in STYLE if j != 3) == 0)
m.Equation(sum(x[1][j] for j in STYLE if j != 4) == 0)

for i in SHIRT:
    m.Equation(sum(x[i][j] for j in STYLE) >= 2)
for j in STYLE:
    m.Equation(sum(x[i][j] for i in SHIRT) >= 2)
    
m.Equation(sum(x[i][3] for i in SHIRT) <= 5)

for i in [2, 3, 4]:
    m.Equation(x[i][1] >= 2)
    
m.Equation(sum(x[3][j] for j in STYLE) <= 5)
m.Equation(sum(x[i][1] for i in SHIRT) >= 6)


### OPTIMIZING
m.options.SOLVER = 1
# m.solve()
m.solve(disp=False)


### RESULTS
print('Objective: ' + str(-1 * m.options.objfcnval))
for i in SHIRT:
    for j in STYLE:
        val = x[i][j].value[0]
        if abs(val) > 1e-6: # only printing non-zeros
            print('{}_{} : {}'.format(shirt_name[i], style_name[j], val))
          
print('Total cost: {} <= {}'.format(sum([shirt_cost[i] * x[i][j].value[0] for j in STYLE for i in SHIRT]), budget))

Objective: 535.0
Jersey_Jersey Set : 5.0
Sweatshirt_Hoodie : 2.0
Red_Criss Cross : 1.0
Red_Crop Top : 2.0
Red_Shoulder : 1.0
Black_Criss Cross : 1.0
Black_Crop Top : 2.0
Black_Halter : 1.0
White_Crop Top : 2.0
White_Halter : 1.0
White_Shoulder : 1.0
Total cost: 300.0 <= 300
